In [1]:
import openai
from pathlib import Path
import json
import llm2geneset
import time
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer
from statsmodels.stats.multitest import multipletests

aclient = openai.AsyncClient()
client = openai.Client()

In [2]:
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2'], use_stemmer=True)

In [3]:
def flatten_list_of_lists(lst):
    return [item for sublist in lst for item in sublist]

def unflatten_list_of_lists(flat_list, original_structure):
    result = []
    index = 0
    for sublist in original_structure:
        length = len(sublist)
        result.append(flat_list[index:index + length] if length > 0 else [])
        index += length
    return result

In [4]:
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", 
             "WikiPathway_2023_Human"]
models = ["gpt-4o-mini-2024-07-18", "gpt-3.5-turbo-0125", "gpt-4o-2024-08-06"]

output = []
for model in models:
    print(model)
    for lib_name in lib_names:
        print(lib_name)
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
        
        test_sets = gen_res["curated_genesets"]
        test_descr = gen_res["descr"]
        test_descr_cleaned = gen_res["descr_cleaned"]

        gt_embs = llm2geneset.get_embeddings(client, test_descr_cleaned)

        for method in ["llm2geneset", "GSAI"]:
            if method == "GSAI":
                # use GSAI to generate geneset name
                gsai_res = await llm2geneset.gsai_bench(aclient, test_sets, model=model, n_retry=5)
                # use lists of size 1 for names.
                names = [[i['name']]for i in gsai_res]
                in_toks = [i["in_toks"] for i in gsai_res]
                out_toks = [i["out_toks"] for i in gsai_res]
            elif method == "llm2geneset":
                llm2geneset_res = await llm2geneset.gs_proposal_bench(aclient, test_sets, model=model)
                # Extract multiple names meeting significance.
                def res2name(res):
                    df = res["ora_results"]
                    _, pvals_corrected, _, _ = multipletests(df["p_val"], method='bonferroni')
                    df["p_adj"] = pvals_corrected
                    df = df[df["p_adj"] < 0.01]
                    if df.shape[0] == 0:
                        return ["None found."]
                    else: 
                        return df["set_descr"].to_list()            
                names = list(map(res2name, llm2geneset_res))
                in_toks = [i["tot_in_toks"] for i in llm2geneset_res]
                out_toks = [i["tot_out_toks"] for i in llm2geneset_res]

            # Since there can be multiple names per gene set
            # compute cosine similarity for each one. 
            flat_names = flatten_list_of_lists(names)
            flat_name_embs = llm2geneset.get_embeddings(client, flat_names)
            name_emb = unflatten_list_of_lists(flat_name_embs,names)

            # Get best score for significant gene sets and report. 
            for i, ref in enumerate(test_descr_cleaned):
                scores = [scorer.score(ref, n) for n in names[i]] 
                rouge1 = [s['rouge1'].recall for s in scores]
                rouge2 = [s['rouge2'].recall for s in scores]

                csim = [np.dot(gt_embs[i], emb) for emb in name_emb[i]]
                    
                x={"model":model,
                   "library":lib_name,
                   "gt_name":test_descr[i],
                   "gt_name_clean":ref,
                   "name":", ".join(names[i]),
                   "ROUGE1":max(rouge1),
                   "ROUGE2":max(rouge2),  
                   "csim":max(csim),
                   "method": method,
                   "in_toks":in_toks[i],
                   "out_toks": out_toks[i]                   
                  }    
            
                output.append(x)

gpt-4o-mini-2024-07-18
WikiPathway_2023_Human


 60%|██████████████████████████████████████████████████████████████████████████▊                                                 | 483/801 [00:30<00:13, 24.38it/s]

retrying
List 5 biological pathways, biological processes, or cellular components that contain the following genes """MAP3K1,MAP3K2,BAD,RPS6,FRS3,TSC2,FRS2,GRIN1,MAPK14,APC,MAPK10,MARCKS,DOK5,NCF1,SQSTM1,RAF1,RELA,CYFIP1,PRKAA2,CRTC1,FOXO3,PRKAA1,GRIP1,DPYSL2,CDH2,NTF3,AKT1,CFL1,SH2B1,SH2B2,NGFR,STAT5B,MAP2K5,NSF,MAP2K2,PRKCD,MAP2K1,EEF2,EIF2S2,NFKB1,EIF2S1,DLG1,CREB1,PIK3R1,TRAF6,GRB2,GRIA2,GRIA3,RAB3A,GRIA1,IRS1,NCF2,IRS2,EIF4EBP1,ELK1,GRIN2B,SIRPA,KCNN2,PLCG1,STAT5A,JUN,KSR1,BDNF,NFATC4,PTPN11,RHOG,LINGO1,VAV2,SYN1,VAV3,NFKBIA,PTPRF,MAPT,CDK5,CAMK4,CAMK1,DOCK3,GABRB3,GSK3B,SHC3,SHC4,RASGRF1,SHC1,NCAM1,SHC2,PIK3R2,ACACB,RAP1A,CASP3,EIF4E,NTRK2,MEF2A,NTRK3,CDKL5,MEF2C,NTRK1,SORT1,STAT1,CHUK,CSNK2A1,PDPK1,STAT3,KIDINS220,MTOR,NGF,RANBP9,TIAM1,CDC42,PPP2CA,ADAM17,RPS6KB1,CDK5R1,SRC,CAMK2A,CTNNB1,KCNA3,YBX1,RPS6KA5,IKBKB,MAPK9,MAPK8,RPS6KA1,CNR1,RPS6KA3,MAPK3,PTK2B,IGF2BP1,HRAS,MAPK1,RAC1,FYN,RACK1,IKBKG,MAPK7,NCK1,JAK2,NCK2""" with high confidence. Be as specific as possible. List non-o

 75%|█████████████████████████████████████████████████████████████████████████████████████████████▌                              | 604/801 [00:36<00:12, 15.24it/s]

retrying
conf is none
Write a critical analysis of the biological processes performed by this system of interacting proteins.

Base your analysis on prior knowledge available in your training data.  After completing your analysis, propose a brief and
detailed name for the most prominent biological process performed by the system.

After completing your analysis, please also assign a confidence score to the process name you selected.  This score should
follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence, while 1.00 reflects
the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities within
the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that
participate in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but
only a few genes in the system contribute

...

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▉            | 723/801 [00:49<00:05, 13.80it/s]

retrying
name is none
Write a critical analysis of the biological processes performed by this system of interacting proteins.

Base your analysis on prior knowledge available in your training data.  After completing your analysis, propose a brief and
detailed name for the most prominent biological process performed by the system.

After completing your analysis, please also assign a confidence score to the process name you selected.  This score should
follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence, while 1.00 reflects
the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities within
the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that
participate in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but
only a few genes in the system contribute

 95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏      | 757/801 [00:51<00:03, 13.65it/s]

retrying
conf is none
Write a critical analysis of the biological processes performed by this system of interacting proteins.

Base your analysis on prior knowledge available in your training data.  After completing your analysis, propose a brief and
detailed name for the most prominent biological process performed by the system.

After completing your analysis, please also assign a confidence score to the process name you selected.  This score should
follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence, while 1.00 reflects
the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities within
the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that
participate in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but
only a few genes in the system contribute

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [00:58<00:00, 13.60it/s]


In [5]:
df = pd.DataFrame(output)
df.to_csv("outputs/gsai_vs_llm2geneset_outputs.tsv", sep="\t", index=False)